In [90]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [91]:
PATH_TRAIN = "B:\\CV Drone Detection\\scaled\\train"
PATH_TEST = "B:\\CV Drone Detection\\scaled\\test"
PATH_VALID = "B:\\CV Drone Detection\\scaled\\valid"

In [104]:


transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),   # різні масштаби об’єкта
    transforms.RandomHorizontalFlip(),                     # віддзеркалення
    transforms.RandomVerticalFlip(),                       # якщо дрони можуть бути в різних положеннях
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),  # міняє колір
    transforms.RandomRotation(20),                         # трохи повертає
    transforms.RandomAffine(degrees=0, translate=(0.1,0.1)), # зсув
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5), # деформація
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.2)),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])


trainset = torchvision.datasets.ImageFolder(root=PATH_TRAIN, transform=transform)
trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root=PATH_TEST, transform=transform)
testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = trainset.classes
print("Classes:", classes)


Classes: ['drone', 'notAdrone']


In [105]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 29 * 29, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 29 * 29)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [106]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [107]:
for epoch in range(2):  

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999: 
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.579
[2,  2000] loss: 0.199
Finished Training


In [108]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the  test images: %d %%' % (
    100 * correct / total), correct, total)

Accuracy of the network on the  test images: 94 % 931 989


In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])
test_imgs = [r"B:\CV Drone Detection\tasting_samples\drone\agenda-luxcrea-dronepexels-2980564356.jpg", 
             r"B:\CV Drone Detection\tasting_samples\drone\agenda-luxcrea-dronepexels-2980564356.jpg",
             r"B:\CV Drone Detection\tasting_samples\notAdrone\amh1tqh06166z417-2033267517.jpg"]

def model_testing(test_imgs = test_imgs):
    for i in range(len(test_imgs)):
        img = transform(Image.open(test_imgs[i])).unsqueeze(0)
        with torch.no_grad():
            outputs = net(img)
            _, predicted = torch.max(outputs.data, 1)
        print("PREDICTION:", classes[predicted.item()])

model_testing()

PREDICTION: drone
PREDICTION: drone
PREDICTION: drone
